In [30]:
import glob
import os
import subprocess
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mintpy import view, plot_network
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, writefile, time_func, utils as ut, utils0 as ut0
from scipy import signal
from scipy.interpolate import Rbf
from urllib.request import urlretrieve

from matplotlib.backends.backend_pdf import PdfPages

In [31]:
TRE_GNSS_filename = r'C:\Users\bvarugu\Documents\ArcGIS\Projects\InSAR_discussion\Outputs\GNSS_station_network_TRE_Towill.csv'
TRE_GNSS = pd.read_csv(TRE_GNSS_filename,header=0);
TRE_GNSS['site'] = TRE_GNSS['CODE'].str[-4:];
used_sites= list(TRE_GNSS['site'][TRE_GNSS['USED']==1]);
filter_by_TRE = True;

In [32]:
InSAR_ts_filename =  r'C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\timeseries_tropHgt_demErr.h5';
maskfile = os.path.join(os.path.dirname(InSAR_ts_filename),'maskTempCoh.h5');
geometry_filename = os.path.join(os.path.dirname(InSAR_ts_filename),'geometryGeo.h5');
azimuthAngle = readfile.read(geometry_filename,datasetName='azimuthAngle')[0];
incidenceAngle = readfile.read(geometry_filename,datasetName='incidenceAngle')[0];
save_gdal_filename = os.path.join(os.path.dirname(InSAR_ts_filename),'ARIASenA35_39N_Asc_velocity_2022_2024_GNSS.tif');
maskTempCoh = readfile.read(maskfile)[0]
gps_dir = os.path.join(r'C:\Users\bvarugu\Documents\GNSS','GNSS_ESESES');
# get analysis metadata from InSAR velocity file
insar_metadata = readfile.read_attribute(InSAR_ts_filename);
date_list = timeseries(InSAR_ts_filename).get_date_list()
if insar_metadata['REF_DATE'] != date_list[0]:
    print('Reference date not set to first date. Setting it ....');
    iargs = [InSAR_ts_filename, '--ref-date', str(date_list[0])]
    import mintpy.cli.reference_date
    mintpy.cli.reference_date.main(iargs)
insar_ts, insar_metadata = readfile.read(InSAR_ts_filename);
lat_step = float(insar_metadata['Y_STEP'])
lon_step = float(insar_metadata['X_STEP'])
(S,N,W,E) = ut.four_corners(insar_metadata)
start_date = insar_metadata.get('START_DATE', None)
end_date = insar_metadata.get('END_DATE', None)
start_date_gnss = dt.datetime.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.datetime.strptime(end_date, "%Y%m%d")
insar_ts = readfile.read(InSAR_ts_filename, datasetName='timeseries')[0]


Reference date not set to first date. Setting it ....
input reference date: 20211106
--------------------------------------------------
change reference date for file: C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\timeseries_tropHgt_demErr.h5
split along y dimension (2188) into 2 boxes
    with each box up to 1094 in y dimension

------- processing patch 1 out of 2 --------------
box width:  4017
box length: 1094
reading data ...
referencing in time ...
--------------------------------------------------
open  HDF5 file C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\timeseries_tropHgt_demErr.h5 in r+ mode
writing dataset /timeseries                block: (0, 82, 0, 1094, 0, 4017)
close HDF5 file C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\timeseries_tropHgt_demErr.h5.

------- processing patch 2 out of 2 --------------
box width:  4017
box length: 1094
reading data ...
referencing in time ...
--------------------------------------------------
open  HDF5 file C:\

In [33]:
gnss_source = 'ESESES';

if filter_by_TRE==True:
    gnss_names,gnss_lats, gnss_lons = gnss.search_gnss(SNWE=(S,N,W,E), source=gnss_source,start_date=start_date, end_date=end_date);
    in_TRE = [i for i,gnss in enumerate(gnss_names) if gnss in used_sites];
    site_names, site_lats, site_lons = gnss_names[in_TRE],gnss_lats[in_TRE],gnss_lons[in_TRE]
else:
    site_names, site_lats, site_lons = gnss.search_gnss(SNWE=(S,N,W,E), source=gnss_source,start_date=start_date, end_date=end_date);

site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)


site_Y,site_X = ut.coordinate(insar_metadata).geo2radar(site_lats, site_lons)[0:2];
site_azimuthAngles = [azimuthAngle[row,col] for row, col in zip(site_Y,site_X)];
site_incidenceAngles = [incidenceAngle[row,col] for row, col in zip(site_Y,site_X)];

load 3655 GNSS sites with fields: site lon lat
keep sites within SNWE of (38.34165133, 40.164983934, -124.415783567, -121.068284906): [66]
['CHO1' 'CHO5' 'DIXN' 'GASB' 'HOPB' 'LNC1' 'LNC2' 'MNRC' 'ORVB' 'P059'
 'P156' 'P164' 'P182' 'P184' 'P185' 'P186' 'P187' 'P188' 'P189' 'P190'
 'P192' 'P195' 'P197' 'P201' 'P202' 'P203' 'P204' 'P205' 'P206' 'P207'
 'P208' 'P263' 'P264' 'P265' 'P267' 'P268' 'P269' 'P270' 'P271' 'P272'
 'P276' 'P312' 'P313' 'P314' 'P315' 'P317' 'P318' 'P319' 'P320' 'P321'
 'P322' 'P329' 'P333' 'P334' 'P335' 'P336' 'P339' 'P340' 'P344' 'P794'
 'PLMO' 'PLSB' 'SUTB' 'TMB2' 'UCD1' 'VCVL']
Initial list of 33 stations used in analysis:
['DIXN', 'HOPB', 'MNRC', 'ORVB', 'P059', 'P182', 'P184', 'P185', 'P189', 'P190', 'P192', 'P197', 'P201', 'P202', 'P203', 'P204', 'P205', 'P206', 'P207', 'P208', 'P264', 'P265', 'P268', 'P271', 'P272', 'P276', 'P322', 'P335', 'P336', 'P339', 'P344', 'UCD1', 'VCVL']


In [34]:
def standardize_longitude(lon, limit='-180to180'):
    """Normalize the longitude value range into (-180, 180] or [0, 360).

    Parameters: lon   - float / np.ndarray, longitude in degree
                limit - str, -180to180 or 0to360
    Returns:    lon   - float / np.ndarray, longitude in degree
    """
    lon = np.asarray(lon)

    # ensure data within (-180, 360)
    lon = np.where(lon >= 360, lon - 360, lon)
    lon = np.where(lon <= -180, lon + 360, lon)

    # range option 1: ensure data within (-180, 180]
    if limit == '-180to180':
        lon = np.where(lon > 180, lon - 360, lon)

    # range option 2: ensure data within [0, 360)
    elif limit == '0to360' and np.nanmin(lon) < 0:
        lon = np.where(lon < 0, lon + 360, lon)

    return float(lon) if np.isscalar(lon) else lon
    
class GNSS_ESESES:
    """GNSS child class for daily solutions processed for the Enhanced Solid
    Earth Science ESDR System (ESESES) project by JPL and SOPAC.

    Website: https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/ESESES_products.html
             http://garner.ucsd.edu/pub/measuresESESES_products/
    """
    def __init__(self, site: str, data_dir=None, version='IGS14', url_prefix=None):
        self.site = site
        self.data_dir = data_dir or os.getcwd()  # Default to current working directory
        self.version = version
        self.source = 'ESESES'
        self.url_prefix = url_prefix
        

        # get file
        self.file = os.path.join(self.data_dir, f'{self.site.lower():s}CleanTrend.neu.Z')

        # get url
        # moved to GNSS_ESESES.dload_site() to avoid searching url_prefix
        # when downloading is not needed.


    def dload_site(self, overwrite=True, total_tries=5, print_msg=True):
        """Download GNSS data file.
        """
        from zipfile import ZipFile

        # get url
        if not self.url_prefix:
            self.url_prefix = gnss.get_ESESES_url_prefix()
        self.url = os.path.join(self.url_prefix, os.path.basename(self.file)).replace("\\", "/");#print('URL is',self.url);
        if self.url and overwrite or not os.path.isfile(self.file):
            print(f"downloading site {self.site:s} from {self.source} to {self.file:s}")
            # retry on download fail
            # https://stackoverflow.com/questions/31529151
            remain_tries = total_tries
            while remain_tries > 0 :
                try:
                    urlretrieve(self.url, self.file)
                    print(f'successfully downloaded: {self.url}')
                except:
                    print(f'error downloading {self.url} on trial no. {total_tries-remain_tries}')
                    remain_tries -= 1
                    continue
                else:
                    break

        # call parent class to download
        #super().dload_site(overwrite=overwrite, print_msg=print_msg)

        # uncompress the downloaded *.z file [for ESESES only]
        with ZipFile(self.file, 'r') as fz:
            fz.extractall(self.data_dir)
        self.file = self.file.strip('.Z')    # update file name
        if print_msg:
            print(f'... extracted to {self.file:s}')

        return self.file


    def get_site_lat_lon(self, print_msg=False) -> (float, float):
        """Get station lat/lon based on processing source.
        Retrieve data from the displacement file.

        Modifies:   self.lat/lon - float
        Returns:    self.lat/lon - float
        """
        # download file if it does not exist
        if not os.path.isfile(self.file):
            self.dload_site(print_msg=print_msg)

        # use the uncompressed data file
        if self.file.endswith('.Z'):
            self.file = self.file[:-2]

        with open(self.file) as f:
            lines = f.readlines()

            # latitude
            lat_line = [x for x in lines if x.startswith('# Latitude')][0].strip('\n')
            self.site_lat = float(lat_line.split()[-1])

            # longitude
            lon_line = [x for x in lines if x.startswith('# East Longitude')][0].strip('\n')
            self.site_lon = float(lon_line.split()[-1])

        # ensure longitude in the range of (-180, 180]
        self.site_lon = standardize_longitude(self.site_lon, limit='-180to180')

        return self.site_lat, self.site_lon
    def _crop_to_date_range(self, start_date: str, end_date: str):
        """Crop the time-series given the start/end_date in format YYYYMMDD,
        and create date_list from dates.
        """
        flag = np.ones(len(self.dates), dtype=bool)
        if start_date:
            t0 = ptime.date_list2vector([start_date])[0][0]
            flag[self.dates < t0] = 0
        if end_date:
            t1 = ptime.date_list2vector([end_date])[0][0]
            flag[self.dates > t1] = 0

        self.dates = self.dates[flag]
        self.dis_e = self.dis_e[flag]
        self.dis_n = self.dis_n[flag]
        self.dis_u = self.dis_u[flag]
        self.std_e = self.std_e[flag]
        self.std_n = self.std_n[flag]
        self.std_u = self.std_u[flag]

        # create member var: date_list
        self.date_list = [x.strftime('%Y%m%d') for x in self.dates]


    def read_displacement(self, start_date=None, end_date=None, print_msg=True, display=False):
        """Read GNSS displacement time-series (defined by start/end_date).

        Parameters: start/end_date - str, date in YYYYMMDD format
        Returns:    dates          - 1D np.ndarray of datetime.datetime object
                    dis_e/n/u      - 1D np.ndarray of displacement in meters in float32
                    std_e/n/u      - 1D np.ndarray of displacement STD in meters in float32
        """
        vprint = print if print_msg else lambda *args, **kwargs: None

        # download file if it does not exist
        if not os.path.isfile(self.file):
            self.dload_site(print_msg=print_msg)

        # use the uncompressed data file
        if self.file.endswith('.Z'):
            self.file = self.file[:-2]

        # read data file
        # use the first 9 cols only, as some epoches miss 10-13 cols: CorrNE/NU/EU, Chi-Squared
        vprint('reading time and displacement in east/north/vertical direction')
        fc = np.loadtxt(self.file, usecols=tuple(range(0,9)))
        num_solution = fc.shape[0]

        # parse dates
        dates = [dt.datetime(int(fc[i, 1]), 1, 1) + dt.timedelta(days=int(fc[i, 2]))
                 for i in range(num_solution)]
        self.dates = np.array(dates)

        # parse displacement data
        (self.dis_n,
         self.dis_e,
         self.dis_u,
         self.std_n,
         self.std_e,
         self.std_u) = fc[:, 3:9].astype(np.float32).T / 1000

        # cut out the specified time range
        self._crop_to_date_range(start_date, end_date)

        # display if requested
        if display:
            self.plot()

        return (self.dates,
                self.dis_e, self.dis_n, self.dis_u,
                self.std_e, self.std_n, self.std_u)



gps_plot_dir = os.path.join(gps_dir,'GNSS_plot');
os.makedirs(gps_plot_dir,exist_ok=True)


In [35]:
wind=2;  #Window for InSAR velocity
# # Get date list

num_date = len(date_list)
date0, date1 = date_list[0], date_list[-1]
insar_dates = ptime.date_list2vector(date_list)[0]


site_displacements = np.zeros([len(site_names),len(insar_dates)]);
site_insar_displacements = np.zeros([len(site_names),len(insar_dates)]);
for i, site_name in enumerate(site_names):
    gnss_obj = GNSS_ESESES(site =site_name,data_dir=gps_dir);gnss_obj.read_displacement();
    #gnss_obj._crop_to_date_range(start_date= date0, end_date= date1);
    gnss_obj_dis_los = ut.enu2los(gnss_obj.dis_e,gnss_obj.dis_n, gnss_obj.dis_u, site_incidenceAngles[i], site_azimuthAngles[i]);
    y, x = int(site_Y[i]),int(site_X[i]);
    for j in range(len(insar_dates)):
        insar_disp = insar_ts[j];
        # desired_index = np.where((gnss_obj.dates > date - dt.timedelta(days=3)) &
        #                      (gnss_obj.dates < date + dt.timedelta(days=3)))[0]
        # site_displacements[i,j]= np.nanmean(gnss_obj_dis_los[desired_index]);
        desired_index = np.where(gnss_obj.dates == insar_dates[j])[0];
        try:
            site_displacements[i,j]= gnss_obj_dis_los[desired_index][0];
        except:
            site_displacements[i,j]= np.nan;
        site_insar_displacements[i,j] = np.nanmean(insar_disp[y-wind:y+wind, x-wind:x+wind],axis=(0,1));
        


reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
reading time and displacement in east/north/vertical direction
searching for ESESES url_prefix ...
http://garner.ucsd.edu/pub/measuresESESES_products/Timeseries/CurrentUntarred/Clean_TrendNeuTimeSeries_comb_20241224 [no]
http://garner.ucsd.edu/pub/measuresESESES_products/Timeseries/CurrentUntarred/Clean_TrendNeuTimeSeries_comb_20241223 [no]
http://garner.ucsd.edu/pub/measuresESESES_products/Timeseries/CurrentUntarred/Clean_TrendNeuTimeSeries_comb_20241222 [no]
http://garner.ucsd.edu/pub/measuresESESES_products/Timeseries/CurrentUntarred/Clean_TrendNeuTimeSeries_comb_20241221 [YES!]
downloading site ORVB from ESESES to C:\Users\bvarugu\Documents\GNSS\GNSS_ESESES\orvbCleanTrend.neu.Z
successfully downloaded: http://garner.ucsd.edu/pub/measuresESESES_products/Timeseries/CurrentUntarred/Clean_TrendNeuTimeSeries_comb_20241221/orvbCleanTrend.neu.Z
... extracted to C:\

In [36]:
for i in range(len(insar_dates)):
    if np.isnan(site_displacements[:,i]).all() == True:
        print('replacing values for {} from {}'.format(insar_dates[i],insar_dates[i-1]))
        site_displacements[:,i] = site_displacements[:,i-1];

In [37]:
def get_surface(station_names,station_Y,station_X,station_velocities,station_insar_velocites,insar_velocity):
    station_names = np.array(station_names);
    station_Y = np.array(station_Y);
    station_X = np.array(station_X);
    median_velocity = np.nanmedian(station_velocities)
    print('Remove NaN Values and velcoties grater than median velocity:{}'.format(median_velocity))
    valid_points = (~np.isnan(station_velocities)) & (~np.isnan(station_insar_velocites))
    station_X_valid = station_X[valid_points]
    station_Y_valid = station_Y[valid_points]
    station_velocities_valid = station_velocities[valid_points];
    station_insar_velocities_valid = station_insar_velocites[valid_points];
    station_names_valid = station_names[valid_points];
    
    print('Remove Duplicates')
    # Combine station_X_valid and station_Y_valid into a single array of coordinates
    coords = np.array(list(zip(station_X_valid, station_Y_valid)))
    
    # Find unique coordinates and their indices
    _, unique_indices = np.unique(coords, axis=0, return_index=True)
    
    # Use these indices to filter station_X_valid, station_Y_valid, station_velocities_valid, and station_names_valid
    station_X_unique = station_X_valid[unique_indices]
    station_Y_unique = station_Y_valid[unique_indices]
    station_velocities_unique = station_velocities_valid[unique_indices]
    station_insar_velocities_unique = station_insar_velocities_valid[unique_indices]
    station_names_unique = station_names_valid[unique_indices]
    station_residual_velocities_unique = station_velocities_unique-station_insar_velocities_unique
    rbf_interpolator = Rbf(station_X_unique, station_Y_unique, station_residual_velocities_unique, function='linear')
    rbf_interpolator2 = Rbf(station_X_unique, station_Y_unique, station_velocities_unique, function='linear')
    # Define grid for interpolation (extend beyond the data range)
    length, width = np.shape(insar_velocity)
    grid_Y, grid_X = np.arange(0, length), np.arange(0,width)
    grid_X, grid_Y = np.meshgrid(grid_X, grid_Y)
    
    # Interpolate on the extended grid
    z_grid_extended = rbf_interpolator(grid_X, grid_Y)
    gnss_vel_grid = rbf_interpolator2(grid_X, grid_Y);
    print('Done')
    return z_grid_extended,gnss_vel_grid


In [38]:
gnss_ts = np.zeros(np.shape(insar_ts), dtype=np.float32)
residual_ts = np.zeros(np.shape(insar_ts), dtype=np.float32)
site_displacements_referenced = site_displacements - site_displacements[:, [0]]
for k,date in enumerate(insar_dates):
    if k !=0:
        print(date)
        residual_ts[k,:,:],gnss_ts[k,:,:]  = get_surface(site_names,site_Y,site_X,site_displacements_referenced[:,k],site_insar_displacements[:,k],
                                                        insar_ts[0]);


2021-11-18 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.0016409270465373993
Remove Duplicates
Done
2021-11-30 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.0031917765736579895
Remove Duplicates
Done
2021-12-12 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.004198230803012848
Remove Duplicates
Done
2022-04-05 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.008572164922952652
Remove Duplicates
Done
2022-04-17 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.002594888210296631
Remove Duplicates
Done
2022-04-29 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.001648621866479516
Remove Duplicates
Done
2022-05-11 00:00:00
Remove NaN Values and velcoties grater than median velocity:-0.0016478421166539192
Remove Duplicates
Done
2022-05-23 00:00:00
Remove NaN Values and velcoties grater than median velocity:0.0024733515456318855
Remove Duplicates
Done
202

In [39]:
def get_new_reference(cummulative_disp):
    # Mask invalid values as NaN
    cummulative_disp[maskTempCoh == 0] = np.nan
    
    # Find the minimum absolute value ignoring NaNs
    min_value = np.nanmin(abs(cummulative_disp))
    
    # Find the index of the minimum value
    index = np.unravel_index(np.nanargmin(abs(cummulative_disp)), cummulative_disp.shape)
    
    print("Minimum Value:", min_value)
    print("Index of Minimum Value:", index)
    return index



In [40]:
#Add residual to the InSAR timeseries
corrected_ts = insar_ts + residual_ts;
cummulative_disp = corrected_ts[-1].copy()
(new_ref_y,new_ref_x) = get_new_reference(cummulative_disp);
print('New reference point is:',new_ref_y,new_ref_x);
insar_metadata['REF_Y'] = str(new_ref_y);
insar_metadata['REF_X'] = str(new_ref_x);
#define file names
date_list = np.array(date_list, dtype=np.bytes_)
residual_ts_outname = os.path.join(os.path.dirname(InSAR_ts_filename),'residual_ts.h5');
gnss_ts_outname = os.path.join(os.path.dirname(InSAR_ts_filename),'gnss_ts.h5');
corrected_ts_outname = os.path.join(os.path.dirname(InSAR_ts_filename),'corrected_ts_TRE_GNSS.h5');
#make a dict to write mintpy timeseries file
residual_ts_dict = {
    "date"       : date_list,
    "timeseries" : residual_ts,
}
gnss_ts_dict = {
    "date"       : date_list,
    "timeseries" : gnss_ts,
}
corrected_ts_dict = {
    "date"       : date_list,
    "timeseries" : corrected_ts,
}
#writefile.write(residual_ts_dict,residual_ts_outname,metadata=insar_metadata);
#writefile.write(gnss_ts_dict,gnss_ts_outname,metadata=insar_metadata);
writefile.write(corrected_ts_dict,corrected_ts_outname,metadata=insar_metadata)

Minimum Value: 1.3969839e-09
Index of Minimum Value: (148, 2233)
New reference point is: 148 2233
create HDF5 file: C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\corrected_ts_TRE_GNSS.h5 with w mode
create dataset /date       of |S8        in size of (82,)                with compression=None
create dataset /timeseries of float32    in size of (82, 2188, 4017)     with compression=None
finished writing to C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\corrected_ts_TRE_GNSS.h5


'C:\\Users\\bvarugu\\Documents\\ARIASenA35\\track39W\\mintpy\\corrected_ts_TRE_GNSS.h5'

In [41]:
corrected_vel_outname = os.path.join(os.path.dirname(InSAR_ts_filename),'velocity_GNSS_integrated.h5');
iargs = [corrected_ts_outname, '-o', corrected_vel_outname, '--update']
import mintpy.cli.timeseries2velocity
mintpy.cli.timeseries2velocity.main(iargs)

update mode: ON
1) output file C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated.h5 NOT found.
run or skip: run.
open timeseries file: corrected_ts_TRE_GNSS.h5
--------------------------------------------------
dates from input file: 82
['20211106', '20211118', '20211130', '20211212', '20220405', '20220417', '20220429', '20220511', '20220523', '20220604', '20220616', '20220628', '20220722', '20220803', '20220815', '20220827', '20220908', '20220920', '20221002', '20221014', '20221026', '20221107', '20221119', '20221201', '20221213', '20221225', '20230106', '20230118', '20230130', '20230211', '20230223', '20230307', '20230319', '20230331', '20230412', '20230424', '20230506', '20230518', '20230530', '20230611', '20230623', '20230705', '20230717', '20230729', '20230810', '20230822', '20230903', '20230915', '20230927', '20231009', '20231021', '20231102', '20231114', '20231126', '20231208', '20231220', '20240101', '20240113', '20240125', '20240206', '20240218', '

In [42]:
corrected_vel_masked_outname = os.path.join(os.path.dirname(InSAR_ts_filename),'velocity_GNSS_integrated_msk.h5');
iargs = [corrected_vel_outname, '-m', maskfile, '-o', corrected_vel_masked_outname]
import mintpy.cli.mask
mintpy.cli.mask.main(iargs)

masking velocity     from C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated.h5 ...
masking velocityStd  from C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated.h5 ...
masking intercept    from C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated.h5 ...
masking interceptStd from C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated.h5 ...
masking residue      from C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated.h5 ...
create HDF5 file: C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy\velocity_GNSS_integrated_msk.h5 with w mode
create dataset /velocity     of float32    in size of (2188, 4017)         with compression=None
create dataset /velocityStd  of float32    in size of (2188, 4017)         with compression=None
create dataset /intercept    of float32    in size of (2188, 4017)         with compression=None
create dataset /interceptStd of f

In [43]:

iargs = [corrected_vel_masked_outname, '-o', save_gdal_filename];
import mintpy.cli.save_gdal
try:
    mintpy.cli.save_gdal.main(iargs)
except:
    print('save_gdal failed.')
    print('cd ',os.path.dirname(corrected_vel_masked_outname))
    print('save_gdal.py '+os.path.basename(corrected_vel_masked_outname)+' -o '+os.path.basename(save_gdal_filename))

save_gdal failed.
cd  C:\Users\bvarugu\Documents\ARIASenA35\track39W\mintpy
save_gdal.py velocity_GNSS_integrated_msk.h5 -o ARIASenA35_39N_Asc_velocity_2022_2024_GNSS.tif
